In [6]:
import pandas as pd
df = pd.read_csv(r'C:\Users\parul.kakade\Documents\NIFTY_Prediction\6.SVM\Data\merged_big_1.csv')
df

,Date,Open,High,Low,Close,SMA_14,EMA_14,RSI_14,ATR_14,STOCH_K,...,SPX_Open,SPX_Volume,VIX_Open,VIX_High,VIX_Low,VIX_Close,expiry,put_ratio,call_ratio,days_to_expiry
0,2023-01-02,18131.70,18215.15,18086.50,18197.45,18217.596429,18223.571572,43.264481,208.520408,45.907686,...,NaN,NaN,14.87,15.45,14.57,14.69,2023-01-05,-0.007264,0.003767,3.0
1,2023-01-03,18163.20,18251.95,18149.80,18232.55,18187.042857,18224.768696,45.155473,200.922522,52.159563,...,3853.290039,3.959140e+09,14.69,14.99,14.31,14.39,2023-01-05,0.002026,0.002026,2.0
2,2023-01-04,18230.65,18243.00,18020.60,18042.95,18160.475000,18200.526203,37.822207,202.456627,38.435131,...,3840.360107,4.414080e+09,14.39,15.68,14.33,15.20,2023-01-05,-0.001681,-0.001681,1.0
3,2023-01-05,18101.95,18120.30,17892.60,17992.15,18140.700000,18172.742709,36.129213,204.259725,31.168645,...,3839.739990,3.893450e+09,15.20,15.60,14.81,14.98,2023-01-05,-0.011156,0.005417,0.0
4,2023-01-06,18008.05,18047.40,17795.55,17859.45,18100.628571,18130.970348,32.088566,207.659031,12.187098,...,3823.370117,3.923560e+09,14.98,15.44,14.62,15.03,2023-01-12,-0.000447,-0.000447,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,2025-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,2025-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703,2025-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,2025-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTER

def predict_close_direction_from_open(df, n_splits=5, pca_var=0.95, smoter_k=5):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    feat = [c for c in df.select_dtypes(include=[np.number]).columns if c != 'Close']
    lag = df[feat].shift(1); lag['Open'] = df['Open']; lag['Close'] = df['Close']; lag['Date'] = df['Date']
    lag = lag.dropna().reset_index(drop=True)
    X, y, dates, opens = lag.drop(columns=['Date','Close']).values, lag['Close'].values, lag['Date'].values, lag['Open'].values
    dirs = (y > opens).astype(int)
    smoter = SMOTER(random_state=42, k_neighbors=smoter_k)
    X_res, y_res = smoter.fit_resample(X, y)
    dirs_res = (y_res > X_res[:, list(lag.drop(columns=['Date','Close']).columns).index('Open')]).astype(int)
    dates_res = np.tile(dates, int(len(y_res)/len(y)))[:len(y_res)]
    opens_res = X_res[:, list(lag.drop(columns=['Date','Close']).columns).index('Open')]
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    preds = np.full_like(y_res, np.nan, dtype=float)
    for tr, te in skf.split(X_res, dirs_res):
        pipe = Pipeline([('s', StandardScaler()), ('p', PCA(n_components=pca_var)), ('m', SVR())])
        pipe.fit(X_res[tr], y_res[tr])
        preds[te] = pipe.predict(X_res[te])
    df_out = pd.DataFrame({'Date':dates_res, 'Open':opens_res, 'Actual_Close':y_res, 'Predicted_Close':preds})
    df_out['Actual_Direction'] = np.where(df_out['Actual_Close']>df_out['Open'],'Up','Down')
    df_out['Predicted_Direction'] = np.where(df_out['Predicted_Close']>df_out['Open'],'Up','Down')
    return {
        'MAE':mean_absolute_error(df_out['Actual_Close'],df_out['Predicted_Close']),
        'RMSE':np.sqrt(mean_squared_error(df_out['Actual_Close'],df_out['Predicted_Close'])),
        'R2':r2_score(df_out['Actual_Close'],df_out['Predicted_Close']),
        'Direction_Accuracy':accuracy_score(df_out['Actual_Direction'],df_out['Predicted_Direction']),
        'Classification_Report':classification_report(df_out['Actual_Direction'],df_out['Predicted_Direction'],digits=4),
        'Confusion_Matrix':confusion_matrix(df_out['Actual_Direction'],df_out['Predicted_Direction'],labels=['Up','Down']),
        'Predictions_DF':df_out
    }
df = pd.read_csv(r'C:\Users\parul.kakade\Documents\NIFTY_Prediction\6.SVM\Data\merged_big_1.csv')
predict_close_direction_from_open(df)

ImportError: cannot import name 'SMOTER' from 'imblearn.over_sampling' (c:\Users\parul.kakade\Documents\NIFTY_Prediction\6.SVM\.venv\Lib\site-packages\imblearn\over_sampling\__init__.py)